# Huge datasets, datasets hiérarchiques

Plan

- [Introduction](#intro)

<h3 id="intro">Introduction</h3>

L'exemple [Building a huge numpy array using pytables](http://stackoverflow.com/questions/8642626/building-a-huge-numpy-array-using-pytables) montre créer une grande matrice qui ne tient pourtant pas en mémoire. Le module [h5py](http://www.h5py.org/) est un module qui permet d'agréger un grand nombre de données dans un seul fichier et de les nommer comme des fichiers sur un disque. L'exemple suivant crée un seul fichier contenant deux tableaux :

In [1]:
import h5py
import random
hf = h5py.File('example.hdf5','w')
arr = [ random.randint(0,100) for h in range(0,1000) ]
hf["random/f0_100"] = arr
arr = [ random.randint(0,1000) for h in range(0,10000) ]
hf["random/f0_1000"] = arr
hf.close()

hf = h5py.File('example.hdf5','r')
print(hf)
for k in hf :
    for k2 in hf[k] :
        obj =hf["{0}/{1}".format(k,k2)]
        print(k, k2, obj, obj.value.shape)
hf.close()

<HDF5 file "example.hdf5" (mode r)>
random f0_100 <HDF5 dataset "f0_100": shape (1000,), type "<i4"> (1000,)
random f0_1000 <HDF5 dataset "f0_1000": shape (10000,), type "<i4"> (10000,)


L'avantage est de pouvoir accéder à une partie d'un ensemble sans que celui-ci ne soit chargé en mémoire :

In [2]:
hf = h5py.File('example.hdf5','r')
print(hf["random/f0_1000"][20:25])
hf.close()

[806  47 605 126 440]


Les deux autres modules intéressants sont [pytables](http://pytables.github.io/) et [blosc](http://www.blosc.org/).